In [5]:
import poseidon_core

In [6]:
# Assign the multi-line ASCII art to a variable using triple quotes
# The 'r' before the quotes makes it a "raw string"
ascii_art = r"""

 ███████████     ███████     █████████  ██████████ █████ ██████████      ███████    ██████   █████
▒▒███▒▒▒▒▒███  ███▒▒▒▒▒███  ███▒▒▒▒▒███▒▒███▒▒▒▒▒█▒▒███ ▒▒███▒▒▒▒███   ███▒▒▒▒▒███ ▒▒██████ ▒▒███ 
 ▒███    ▒███ ███     ▒▒███▒███    ▒▒▒  ▒███  █ ▒  ▒███  ▒███   ▒▒███ ███     ▒▒███ ▒███▒███ ▒███ 
 ▒██████████ ▒███      ▒███▒▒█████████  ▒██████    ▒███  ▒███    ▒███▒███      ▒███ ▒███▒▒███▒███ 
 ▒███▒▒▒▒▒▒  ▒███      ▒███ ▒▒▒▒▒▒▒▒███ ▒███▒▒█    ▒███  ▒███    ▒███▒███      ▒███ ▒███ ▒▒██████ 
 ▒███        ▒▒███     ███  ███    ▒███ ▒███ ▒   █ ▒███  ▒███    ███ ▒▒███     ███  ▒███  ▒▒█████ 
 █████        ▒▒▒███████▒  ▒▒█████████  ██████████ █████ ██████████   ▒▒▒███████▒   █████  ▒▒█████
▒▒▒▒▒           ▒▒▒▒▒▒▒     ▒▒▒▒▒▒▒▒▒  ▒▒▒▒▒▒▒▒▒▒ ▒▒▒▒▒ ▒▒▒▒▒▒▒▒▒▒      ▒▒▒▒▒▒▒    ▒▒▒▒▒    ▒▒▒▒▒ 
                                                                                                  
                                                                                                  
                                                                                                                                                                                                                                                  
             
"""

# Print the variable to the terminal
print(ascii_art)



 ███████████     ███████     █████████  ██████████ █████ ██████████      ███████    ██████   █████
▒▒███▒▒▒▒▒███  ███▒▒▒▒▒███  ███▒▒▒▒▒███▒▒███▒▒▒▒▒█▒▒███ ▒▒███▒▒▒▒███   ███▒▒▒▒▒███ ▒▒██████ ▒▒███ 
 ▒███    ▒███ ███     ▒▒███▒███    ▒▒▒  ▒███  █ ▒  ▒███  ▒███   ▒▒███ ███     ▒▒███ ▒███▒███ ▒███ 
 ▒██████████ ▒███      ▒███▒▒█████████  ▒██████    ▒███  ▒███    ▒███▒███      ▒███ ▒███▒▒███▒███ 
 ▒███▒▒▒▒▒▒  ▒███      ▒███ ▒▒▒▒▒▒▒▒███ ▒███▒▒█    ▒███  ▒███    ▒███▒███      ▒███ ▒███ ▒▒██████ 
 ▒███        ▒▒███     ███  ███    ▒███ ▒███ ▒   █ ▒███  ▒███    ███ ▒▒███     ███  ▒███  ▒▒█████ 
 █████        ▒▒▒███████▒  ▒▒█████████  ██████████ █████ ██████████   ▒▒▒███████▒   █████  ▒▒█████
▒▒▒▒▒           ▒▒▒▒▒▒▒     ▒▒▒▒▒▒▒▒▒  ▒▒▒▒▒▒▒▒▒▒ ▒▒▒▒▒ ▒▒▒▒▒▒▒▒▒▒      ▒▒▒▒▒▒▒    ▒▒▒▒▒    ▒▒▒▒▒ 





